<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Libraries and Indicator Function

!pip install -q bta-lib
!pip install -q ta

import btalib
import numpy as np
import pandas as pd
from ta.trend import PSARIndicator
from ta.momentum import WilliamsRIndicator
from ta.trend import AroonIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.trend import CCIIndicator
from ta.momentum import ROCIndicator
from ta.trend import ADXIndicator
from ta.momentum import ultimate_oscillator
from ta.volume import ChaikinMoneyFlowIndicator
from ta.trend import KSTIndicator
from ta.momentum import TSIIndicator
import yfinance as yf
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

warnings.filterwarnings("ignore")


def create_indicators(data):
  data["RSI"] = btalib.rsi(data["Close"], period=14).df

  data["STOCH-K"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["k"]
  data["STOCH-D"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["d"]

  data["WILLIAMS"] = WilliamsRIndicator(data["High"], data["Low"], data["Close"]).williams_r()
  data["AROON"] = AroonIndicator(close=data["Close"], window=25).aroon_indicator()
  data['CCI'] = CCIIndicator(close=data['Close'], low=data["Low"], high=data["High"], window=14).cci()
  data['ROC'] = ROCIndicator(close=data['Close'], window=5).roc()

  adx_indicator = ADXIndicator(high=data['High'], low=data['Low'], close=data['Close'], window=14)
  data['ADX'] = adx_indicator.adx()
  data['+DI'] = adx_indicator.adx_pos()
  data['-DI'] = adx_indicator.adx_neg()

  data['ULTIMATE-OSC'] = ultimate_oscillator(high=data['High'], low=data['Low'], close=data['Close'], window1=7, window2=14, window3=28)
  data['MONEY-FLOW'] = ChaikinMoneyFlowIndicator(high=data['High'], low=data['Low'], close=data['Close'], volume=data['Volume'], window=20).chaikin_money_flow()
  data['KST'] = KSTIndicator(data['Close']).kst()

  data['TSI'] = TSIIndicator(data['Close']).tsi()

  data = data.dropna()
  data = data.reset_index()
  return data

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#@title Download the Model

import gdown

gdown.download("https://drive.google.com/u/1/uc?id=1-3r7tu0ZQXWNtqMQ35rX0DexxNt9P1l5&export=download", "/content/", quiet=False)
# gdown.download("https://drive.google.com/u/0/uc?id=117pezAA6jRLCwIsdpEhZgEa9tEanlC0O&export=download", "/content/", quiet=False)

# data = pd.read_csv("data.csv")
model = tf.keras.models.load_model("best_model_2.h5")

Downloading...
From: https://drive.google.com/u/1/uc?id=1-3r7tu0ZQXWNtqMQ35rX0DexxNt9P1l5&export=download
To: /content/best_model_2.h5
100%|██████████| 236M/236M [00:01<00:00, 132MB/s]


In [2]:
#@title Stock List
sheet_id = "1RSqOXkFTAO7g4H9LEY3d3IX6H6bJaYk1"
sheet_name = "Sheet_1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
result_df = pd.read_csv(url)

sheet_id = "1AA9MfqOtAAgO97__aomD79DciyT-PkRQ"
sheet_name = "Sheet_1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
result_df = pd.read_csv(url)

nasdaq100 = ['AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN',
             'NVDA', 'TSLA', 'META', 'AVGO', 'ASML',
             'PEP', 'COST', 'ADBE', 'AZN', 'CSCO',
             'NFLX', 'AMD', 'CMCSA', 'TMUS', 'TXN',
             'QCOM', 'HON', 'INTU', 'INTC', 'SNY',
             'VZ', 'AMGN', 'SBUX', 'ISRG', 'AMAT',
             'BKNG', 'ADI', 'MDLZ', 'PDD', 'GILD',
             'ADP', 'VRTX', 'ABNB', 'LRCX', 'PYPL',
             'REGN', 'EQIX', 'MU', 'CSX', 'SNPS',
             'CME', 'CDNS', 'KLAC', 'NTES']

In [11]:
data = pd.DataFrame()

for stock in ["AMZN"]:
  stock_df = yf.download(stock, start="2000-01-01", end="2023-01-01", progress=False)
  stock_df = create_indicators(stock_df)
  stock_df["signal"] = 0

  for index, row in stock_df.iterrows():
    if index > 0 and index < stock_df.shape[0]-1 and stock_df["Close"].iloc[index+1] > ((2*stock_df["Close"].iloc[index]/100) + stock_df["Close"].iloc[index]):
      stock_df["signal"].iloc[index] = 1

  stock_df = stock_df.drop(["Date", "Open", "High", "Low", "Close", "Volume", "Adj Close"], axis=1)

  data = pd.concat([data, stock_df], ignore_index=True)

In [12]:
data

,RSI,STOCH-K,STOCH-D,WILLIAMS,AROON,CCI,ROC,ADX,+DI,-DI,ULTIMATE-OSC,signal
0,47.300736,69.469695,74.595956,-35.454542,16.0,45.103588,-3.023067,19.867793,27.729663,23.503735,53.653606,0
1,45.801030,62.424244,69.924241,-41.818185,16.0,16.451525,-0.750001,18.697960,26.872053,25.059569,44.274482,0
2,45.249237,59.545453,63.813131,-44.090913,16.0,-0.375131,-11.203009,17.608107,25.100507,26.888953,50.133482,0
3,42.492667,52.878784,58.282827,-55.454549,16.0,-30.611233,-11.915885,16.596100,24.119256,25.837787,48.246409,0
4,41.038782,46.287878,52.904038,-61.590906,16.0,-72.475723,-9.433962,16.103983,23.108248,28.076473,36.941077,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5754,38.027561,17.632779,13.488271,-78.571429,-64.0,-109.900177,-2.970636,21.166661,17.125746,30.622835,41.966207,0
5755,34.876981,12.690481,14.345135,-94.357136,-72.0,-110.652310,-2.213845,21.673828,16.265949,29.085418,41.380936,0
5756,33.239777,9.321425,13.214895,-99.107162,-76.0,-122.428474,-3.955866,22.464063,15.622566,30.829698,38.861315,1
5757,39.186785,7.879113,9.963673,-82.898363,-76.0,-76.159552,-2.984899,22.705148,17.060491,28.949005,45.840268,0


In [8]:
#@title Data Preprocessing
df = pd.DataFrame(data)

input_columns = df.columns[:data.shape[1]-1]
output_column = "signal"

df[output_column] = df[output_column].astype(int)

X = df[input_columns].values
y = df[output_column].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight("balanced", classes=[0, 1], y=y_train)
class_weight = {cls: weight for cls, weight in zip([0, 1], class_weights)}
class_weight

{0: 0.6007569825110937, 1: 2.981217616580311}

In [ ]:
#@title LSTM Model

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_resampled.shape[1], 1)),

    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('lstm_model.h5', save_best_only=True)

model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=256, validation_split=0.2, class_weight=class_weight,
          callbacks=[early_stopping, model_checkpoint])

In [ ]:
#@title Conv1D model
from tensorflow import keras
from tensorflow.keras import layers

X_train_resampled = np.expand_dims(X_train_resampled, axis=2)
X_test = np.expand_dims(X_test, axis=2)

def create_model():
    input_layer = keras.Input(shape=(X_train_resampled.shape[1], X_train_resampled.shape[2]))

    x = layers.Conv1D(
        filters=32, kernel_size=3, strides=2, activation="relu", padding="same"
    )(input_layer)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=64, kernel_size=3, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=128, kernel_size=5, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=256, kernel_size=5, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=512, kernel_size=7, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=1024, kernel_size=7, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dropout(0.2)(x)

    x = layers.Flatten()(x)

    x = layers.Dense(4096, activation="relu")(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(
        2048, activation="relu", kernel_regularizer=keras.regularizers.L2()
    )(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(
        1024, activation="relu", kernel_regularizer=keras.regularizers.L2()
    )(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(
        128, activation="relu", kernel_regularizer=keras.regularizers.L2()
    )(x)
    output_layer = layers.Dense(1, activation="sigmoid")(x)

    return tf.keras.Model(inputs=input_layer, outputs=output_layer)

model = create_model()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model_2_gpt_deneme.h5', save_best_only=False)

model.fit(X_train_resampled, y_train_resampled, epochs=200, batch_size=512, validation_split=0.2,
          callbacks=[early_stopping, model_checkpoint], class_weight=class_weight)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/200
232/319 [====================>.........] - ETA: 9s - loss: 2.3807 - accuracy: 0.3754

In [ ]:
# model.save("xu030-long-deneme.h5")
model = tf.keras.models.load_model("/content/drive/MyDrive/best_model_2_gpt_deneme.h5")
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

917/917 [==============================] - 10s 8ms/step - loss: 0.5848 - accuracy: 0.8265
Test Loss: 0.584844708442688
Test Accuracy: 0.8264679908752441


In [ ]:
for stock in ["TUPRS.IS"]:
  predictions = []

  stock_data = yf.download(stock, start="2000-01-01", end="2023-08-10", progress=False)
  stock_data = create_indicators(stock_data)

  stock_data["signal"] = 0

  for row in range(stock_data.shape[0]):
    if row+1 != stock_data.shape[0] and stock_data["Close"].iloc[row+1] > stock_data["Close"].iloc[row] + 5*(stock_data["Close"].iloc[row])/100:
      stock_data["signal"].iloc[row] = 1

  stock_data = stock_data.drop(["Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1)

  for index, row in stock_data.iterrows():
      x = row[1:34]

      new_data = x.to_numpy().reshape(1, -1)
      new_data = scaler.transform(new_data)
      prediction = model.predict(new_data, verbose=None)

      # print(f"Prediction for date {row[0]}: {np.round(prediction[0][0])}")

      if np.round(prediction[0][0]) == row["signal"]:
        predictions.append(1)
      elif np.round(prediction[0][0]) != row["signal"]:
        predictions.append(0)

  print(f"{stock} Accuracy: {predictions.count(1) / len(predictions) * 100}")

In [ ]:
buy_stocks = []

for stock in result_df["STOCK"].unique():
  stock_data = yf.download(stock, start="2021-06-01", end="2023-08-17", progress=False)
  stock_data = create_indicators(stock_data)

  change = ((stock_data["Close"].iloc[-1] - stock_data["Close"].iloc[-2]) / stock_data["Close"].iloc[-2])*100
  change = round(change, 2)

  stock_data = stock_data.drop(["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1)

  x = stock_data.iloc[-2]

  new_data = x.to_numpy().reshape(1, -1)
  new_data = scaler.transform(new_data)
  prediction = model.predict(new_data, verbose=None)

  if round(prediction[0][0]*100,2) > 75:
    buy_stocks.append([stock, round(prediction[0][0]*100,2), change])

buy_df = pd.DataFrame(buy_stocks, columns=["stock", "probability", "change"]).sort_values(by="probability", ascending=False)
print(f'Pozitif kapanan hisse sayısı: {buy_df[buy_df["change"] > 0].shape[0]}')
print(f'Negatif kapanan hisse sayısı: {buy_df[buy_df["change"] < 0].shape[0]}')
print(f'Günlük değişim ortalaması: %{round(buy_df["change"].sum()/buy_df.shape[0], 2)}\n')
buy_df

Pozitif kapanan hisse sayısı: 2
Negatif kapanan hisse sayısı: 3
Günlük değişim ortalaması: %-0.94



,stock,probability,change
3,PGSUS.IS,100.00,-1.96
1,GARAN.IS,99.71,-3.15
4,THYAO.IS,96.80,-2.00
0,ENKAI.IS,89.29,1.82
2,KOZAL.IS,77.66,0.60


In [ ]:
2023-08-08
1	BIMAS.IS	98.39	-1.34
4	PGSUS.IS	97.33	9.52
2	GARAN.IS	96.60	-0.43
5	PETKM.IS	90.65	5.20
3	GUBRF.IS	90.02	-0.40
0	ARCLK.IS	83.10	1.14

2023-08-08
2	PGSUS.IS	100.00	0.33
1	GARAN.IS	97.05	3.03
0	BIMAS.IS	96.98	1.54
3	SASA.IS	95.37	-3.57
4	TCELL.IS	93.31	0.94

2023-08-09
0	BIMAS.IS	99.82	9.99
2	PETKM.IS	99.82	0.33
3	THYAO.IS	99.22	-0.69
1	GARAN.IS	92.52	9.91

2023-08-10
2	TOASO.IS	98.16	-4.08
0	GARAN.IS	85.31	2.83
1	SAHOL.IS	76.00	-0.84

2023-08-11
1	KRDMD.IS	100.00	2.44
2	THYAO.IS	97.89	2.79
0	FROTO.IS	90.15	2.24

2023-08-14
3	THYAO.IS	99.93	-1.62
0	AKBNK.IS	99.74	-2.20
1	GARAN.IS	96.38	0.63
2	TAVHL.IS	87.66	5.62

2023-08-15
3	THYAO.IS	100.00	-0.65
0	AKBNK.IS	99.94	0.53
2	SASA.IS	99.93	-0.99
1	GARAN.IS	97.06	-0.63

2023-08-16
1	PGSUS.IS	99.99	-1.96
0	GARAN.IS	99.91	-3.15
2	SASA.IS	99.26	-1.28
3	PETKM.IS	76.96	0.98

2023-08-17
1	PGSUS.IS	100.00	4.82
5	TOASO.IS	100.00	2.20
0	ODAS.IS	99.83	-1.25
6	ISCTR.IS	97.81	0.94
7	YKBNK.IS	94.71	2.77
3	SASA.IS	84.09	0.00
4	TAVHL.IS	81.61	0.39
2	SAHOL.IS	79.27	0.74

2023-08-18
0	EKGYO.IS	99.99	-4.26
3	TOASO.IS	99.95	-3.51
1	GARAN.IS	99.64	-6.55
2	PGSUS.IS	99.56	-4.95
4	ISCTR.IS	96.21	-3.49